In [16]:
import sys
spaceNetUtilitiesPath = '/opt/utilities/python/'
if not spaceNetUtilitiesPath in sys.path:
    print('adding spaceNetUtilitiesPath')
    sys.path.extend([spaceNetUtilitiesPath])
else:
    print('SpaceNet Utilities Path Already Exists')
from spaceNetUtilities import geoTools as gT
from spaceNetUtilities import labelTools as lT



SpaceNet Utilities Path Already Exists


In [17]:
sys.path

['',
 '/opt/utilities/python',
 '/software/spaceNetTensorFlowObj',
 '/opt/models',
 '/opt/models/slim',
 '/usr/lib/python2.7',
 '/usr/lib/python2.7/plat-x86_64-linux-gnu',
 '/usr/lib/python2.7/lib-tk',
 '/usr/lib/python2.7/lib-old',
 '/usr/lib/python2.7/lib-dynload',
 '/usr/local/lib/python2.7/dist-packages',
 '/usr/lib/python2.7/dist-packages',
 '/usr/local/lib/python2.7/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/opt/utilities/python/']

In [18]:
# Copyright 2017 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

r"""Convert raw PASCAL dataset to TFRecord for object_detection.

Example usage:
    ./create_pascal_tf_record --data_dir=/home/user/VOCdevkit \
        --year=VOC2012 \
        --output_path=/home/user/pascal.record
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import hashlib
import io
import logging
import os


from lxml import etree
import PIL.Image
import tensorflow as tf

from object_detection.utils import dataset_util
from object_detection.utils import label_map_util
from tqdm import tqdm


def dict_to_tf_example(data, label_map_dict, img_path=[], dataDir=''):
    """Convert XML derived dict to tf.Example proto.
  
    Notice that this function normalizes the bounding box coordinates provided
    by the raw data.
  
    Args:
      data: dict holding PASCAL XML fields for a single image (obtained by
        running dataset_util.recursive_parse_xml_to_dict)
      dataset_directory: Path to root directory holding PASCAL dataset
      label_map_dict: A map from string label names to integers ids.
      ignore_difficult_instances: Whether to skip difficult instances in the
        dataset  (default: False).
      image_subdirectory: String specifying subdirectory within the
        PASCAL dataset directory holding the actual image data.
  
    Returns:
      example: The converted tf.Example.
  
    Raises:
      ValueError: if the image pointed to by data['filename'] is not a valid JPEG
    """
    if not img_path:
        img_path = data['filename']
    
    with tf.gfile.GFile(img_path, 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = PIL.Image.open(encoded_jpg_io)
    if image.format != 'JPEG':
        raise ValueError('Image format not JPEG')
    key = hashlib.sha256(encoded_jpg).hexdigest()

    width = int(data['size']['width'])
    height = int(data['size']['height'])

    xmin = []
    ymin = []
    xmax = []
    ymax = []
    classes = []
    classes_text = []
    truncated = []
    poses = []
    difficult_obj = []
    
    if 'object' in data:
        for obj in data['object']:
            difficult = bool(int(obj['difficult']))
        

            difficult_obj.append(int(difficult))

            xmin.append(float(obj['bndbox']['xmin']) / width)
            ymin.append(float(obj['bndbox']['ymin']) / height)
            xmax.append(float(obj['bndbox']['xmax']) / width)
            ymax.append(float(obj['bndbox']['ymax']) / height)
            classes_text.append(obj['name'].encode('utf8'))
            classes.append(label_map_dict[obj['name']])
            truncated.append(int(obj['truncated']))
            poses.append(obj['pose'].encode('utf8'))
            #print(xmin)
            #print(ymin)
            #print(xmax)
            #print(ymax)
            example = tf.train.Example(features=tf.train.Features(feature={
                'image/height': dataset_util.int64_feature(height),
                'image/width': dataset_util.int64_feature(width),
                'image/filename': dataset_util.bytes_feature(
                    data['filename'].encode('utf8')),
                'image/source_id': dataset_util.bytes_feature(
                    data['filename'].encode('utf8')),
                'image/key/sha256': dataset_util.bytes_feature(key.encode('utf8')),
                'image/encoded': dataset_util.bytes_feature(encoded_jpg),
                'image/format': dataset_util.bytes_feature('jpeg'.encode('utf8')),
                'image/object/bbox/xmin': dataset_util.float_list_feature(xmin),
                'image/object/bbox/xmax': dataset_util.float_list_feature(xmax),
                'image/object/bbox/ymin': dataset_util.float_list_feature(ymin),
                'image/object/bbox/ymax': dataset_util.float_list_feature(ymax),
                'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
                'image/object/class/label': dataset_util.int64_list_feature(classes),
                'image/object/difficult': dataset_util.int64_list_feature(difficult_obj),
                'image/object/truncated': dataset_util.int64_list_feature(truncated),
                'image/object/view': dataset_util.bytes_list_feature(poses),
            }))
        return example

    else:
        return []

def createTFRecord(txtList, label_map_path, output_dir, dataDir='', maxSamples=-1):
    writer = tf.python_io.TFRecordWriter(output_dir)
    label_map_dict = label_map_util.get_label_map_dict(label_map_path)


    rowList = []
    with open(txtList, 'r') as f:
        for row in f:
            row = row.rstrip()
            rowList.append(row.split(' '))
            
    if maxSamples ==-1:
        maxSamples == len(rowList)
    maxLength = min(len(rowList), maxSamples)
    for row in tqdm(rowList[0:maxLength]):
        imgPath = row[0]
        labelPath = row[1]

        if dataDir != "":
            labelPath = os.path.join(dataDir, os.path.basename(labelPath))
            imgPath = os.path.join(dataDir, os.path.basename(imgPath))
        with tf.gfile.GFile(labelPath, 'r') as fid:
            xml_str = fid.read()
        xml = etree.fromstring(xml_str)
        data = dataset_util.recursive_parse_xml_to_dict(xml)['annotation']

        tf_example = dict_to_tf_example(data, label_map_dict, img_path=imgPath)
        if tf_example:
            #print (row) 
            writer.write(tf_example.SerializeToString())

    writer.close()

    return 'test'


In [19]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import hashlib
import io
import logging
import os

from lxml import etree
import PIL.Image
import tensorflow as tf

from object_detection.utils import dataset_util
from object_detection.utils import label_map_util

In [20]:
filePath = '/data/spacenetV2_TrainData/AOI_2_Vegas_Train_PASCALVOC/trainval.txt'
pathpbTxt = '/data/spacenetV2_TrainData/AOI_2_Vegas_Train_PASCALVOC/data/spacenet_label_map.pbtxt'
output_dir = '/data/spacenetV2_TrainData/AOI_2_Vegas_Train_PASCALVOC/tf_train1.record'
dataDir = '/data/spacenetV2_TrainData/AOI_2_Vegas_Train_PASCALVOC/annotations'
createTFRecord(filePath, pathpbTxt, output_dir, dataDir=dataDir)

filePath = '/data/spacenetV2_TrainData/AOI_2_Vegas_Train_PASCALVOC/test.txt'
output_dir = '/data/spacenetV2_TrainData/AOI_2_Vegas_Train_PASCALVOC/tf_test1.record'
createTFRecord(filePath, pathpbTxt, output_dir, dataDir=dataDir)


#with tf.gfile.GFile(path, 'r') as fid:
#    xml_str = fid.read()
#xml = etree.fromstring(xml_str)
#data = dataset_util.recursive_parse_xml_to_dict(xml)['annotation']

100%|██████████| 3079/3079 [00:28<00:00, 107.10it/s]


'test'

100%|██████████| 3080/3080 [00:43<00:00, 71.07it/s]


'test'

{'filename': '/usr/local/share/data/spacenetV2_TrainData/AOI_2_Vegas_Train_PASCALVOC/RGB-PanSharpen/RGB-PanSharpen__-115.2513576_36.2230676999.tif',
 'folder': 'spacenetV2',
 'object': [{'bndbox': {'xmax': '156',
    'xmin': '51',
    'ymax': '76',
    'ymin': '4'},
   'difficult': '0',
   'name': 'building',
   'pose': 'Left',
   'truncated': '0'},
  {'bndbox': {'xmax': '373', 'xmin': '287', 'ymax': '101', 'ymin': '49'},
   'difficult': '0',
   'name': 'building',
   'pose': 'Left',
   'truncated': '0'},
  {'bndbox': {'xmax': '298', 'xmin': '222', 'ymax': '92', 'ymin': '0'},
   'difficult': '0',
   'name': 'building',
   'pose': 'Left',
   'truncated': '0'},
  {'bndbox': {'xmax': '400', 'xmin': '387', 'ymax': '89', 'ymin': '55'},
   'difficult': '0',
   'name': 'building',
   'pose': 'Left',
   'truncated': '0'},
  {'bndbox': {'xmax': '400', 'xmin': '387', 'ymax': '182', 'ymin': '139'},
   'difficult': '0',
   'name': 'building',
   'pose': 'Left',
   'truncated': '0'},
  {'bndbox': {

{u'building': 1,
 u'cable_distribution_cabinet': 2,
 u'catenary_mast': 3,
 u'compensator': 4,
 u'connection': 5,
 u'generator': 6,
 u'insulator': 7,
 u'none_of_the_above': 0,
 u'plant': 8,
 u'pole': 9,
 u'portal': 10,
 u'station': 11,
 u'sub_station': 12,
 u'substation': 12,
 u'switch': 13,
 u'terminal': 14,
 u'tower': 15,
 u'transformer': 16}